In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans
# Loads data.
dataset = spark.read.csv("hack_data.csv", header=True, inferSchema=True)

In [4]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [5]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       NULL|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       NULL| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [6]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
feat_cols = ['Session_Connection_Time', 
             'Bytes Transferred', 
             'Kali_Trace_Used',
             'Servers_Corrupted', 
             'Pages_Corrupted',
             'WPM_Typing_Speed']

In [9]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [10]:
final_data = vec_assembler.transform(dataset)

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol="features", 
                        outputCol="scaledFeatures", 
                        withStd=True, 
                        withMean=False)

In [13]:
# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(final_data)

In [14]:
# Normalize each feature to have unit standard deviation.
cluster_final_data = scalerModel.transform(final_data)

In [15]:
# kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
# kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [16]:
# model_k3 = kmeans3.fit(cluster_final_data)
# model_k2 = kmeans2.fit(cluster_final_data)

In [17]:
# wssse_k3 = model_k3.computeCost(cluster_final_data)
# wssse_k2 = model_k2.computeCost(cluster_final_data)

Not much to be gained from the WSSSE, after all, we would expect that as K increases, the
WSSSE decreases. We could however continue the analysis by seeing the drop from K=3 to K=4
to check if the clustering favors even or odd numbers. This won't be substantial, but its worth a look:

In [18]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

NameError: name 'model_k3' is not defined

In [ ]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

It was 2 hackers, in fact, our clustering algorithm created two equally
sized clusters with K=2, no way that is a coincidence!